In [2]:
import kaggle

!kaggle datasets download ankitbansal06/retail-orders -f orders.csv


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:01<00:00, 106kB/s]
100%|##########| 200k/200k [00:01<00:00, 106kB/s]


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

In [7]:
#read data from the file and handle null values

data = pd.read_csv("orders.csv", na_values = ['Not Available', 'unknown'])

df = pd.DataFrame(data)
df
df["Ship Mode"].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [8]:
#rename columns names ..make them lower case and replace space with underscore

df.columns = df.columns.str.lower()
df.columns

Index(['order id', 'order date', 'ship mode', 'segment', 'country', 'city',
       'state', 'postal code', 'region', 'category', 'sub category',
       'product id', 'cost price', 'list price', 'quantity',
       'discount percent'],
      dtype='object')

In [9]:
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [11]:
#derive new columns discount , sale price and profit
df

df["discount"] = df["discount_percent"]/100*df["list_price"]
df

df["sale_price"] = df["list_price"] - df["discount"]
df

df["profit"] = df["sale_price"] - df["cost_price"]
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [66]:
#convert order date from object data type to datetime
df["order_date"] = pd.to_datetime(df["order_date"],format = "%Y-%m-%d")

In [68]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [82]:
#drop cost price list price and discount percent columns
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

In [84]:
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1


In [99]:
#load the data into sql server using replace option

import pandas as pd
import mysql.connector

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='shabbir#123',
    database='ecom'
)
cursor = conn.cursor()

# Replace NaN with None to handle SQL NULL
df = df.where(pd.notnull(df), None)

# Clean column names
df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

# Table name where data will be inserted
table_name = 'retail'  # Replace with the actual table name

# Ensure the table exists (optional)
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
cursor.execute(create_table_query)

# Insert DataFrame data into the MySQL table
for _, row in df.iterrows():
    # Convert row to tuple and handle NaN/None explicitly
    values = tuple(None if pd.isna(x) else x for x in row)
    sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
    cursor.execute(sql, values)

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()

print(f"Data from DataFrame has been successfully loaded into the `{table_name}` table.")


Data from DataFrame has been successfully loaded into the `retail` table.


In [2]:
# connection building to MySql

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='shabbir#123',
    database='ecom'
)
cursor = conn.cursor()

In [12]:
# find top 10 highest reveue generating products

query = """select product_id, round(sum(sale_price * quantity), 2) as sales
from retail
group by product_id order by sales desc limit 10;"""

cursor.execute(query)
data = cursor.fetchall()

df = pd.DataFrame(data, columns = ["Products","Revenue"])
df

,Products,Revenue
0,TEC-CO-10004722,245056.0
1,OFF-BI-10000545,163777.7
2,TEC-MA-10002412,130406.4
3,FUR-CH-10002024,120090.7
4,TEC-PH-10001459,113041.9
5,TEC-CO-10001449,107388.0
6,OFF-BI-10003527,97082.9
7,TEC-MA-10000822,89622.3
8,FUR-BO-10002213,84014.8
9,TEC-MA-10001047,81549.0


In [8]:
# find top 5 highest selling products in each region

query = """select * from (select region, product_id, sum(quantity) as sales,
row_number() over(partition by region order by sum(quantity) desc) as rn
from retail
group by region, product_id) as a
where rn <= 5;"""

cursor.execute(query)
data = cursor.fetchall()

df = pd.DataFrame(data, columns = ["Region","Product","Sales","Rank"])
df

,Region,Product,Sales,Rank
0,Central,OFF-BI-10000301,34,1
1,Central,OFF-BI-10000756,33,2
2,Central,OFF-BI-10000546,29,3
3,Central,OFF-BI-10001249,29,4
4,Central,FUR-CH-10002304,27,5
5,East,OFF-PA-10001970,33,1
6,East,OFF-BI-10003656,32,2
7,East,FUR-FU-10004848,31,3
8,East,OFF-FA-10000621,31,4
9,East,OFF-FA-10002780,29,5


In [25]:
# find month over month growth comparison for 2022 and 2023 sales eg : jan 2022 vs jan 2023

query = """select month(order_date) as month, round(sum(sale_price * quantity), 2) as sales
from retail
where year(order_date) = 2022
group by month order by month;"""

cursor.execute(query)
data1 = cursor.fetchall()

df1 = pd.DataFrame(data1, columns = ["Month","2022 Sales"])
df1


query = """select month(order_date) as month, round(sum(sale_price * quantity), 2) as sales
from retail
where year(order_date) = 2023
group by month order by month;"""

cursor.execute(query)
data2 = cursor.fetchall()

df2 = pd.DataFrame(data2, columns = ["Month","2023 Sales"])
df2

df = pd.merge(df1, df2, on = "Month")
df

,Month,2022 Sales,2023 Sales
0,1,437431.3,434765.5
1,2,444011.1,731638.8
2,3,394105.2,393051.9
3,4,476400.9,543231.5
4,5,413625.5,410707.9
5,6,465300.3,328939.0
6,7,375278.4,422533.7
7,8,534562.4,465010.3
8,9,433887.0,420620.5
9,10,601707.8,626498.3


In [5]:
# for each category which month had highest sales 
query = """with b as(select category, months, sales from
(select category, date_format(order_date, "%Y%m") as months, round(sum(sale_price * quantity), 2) as sales,
row_number() over(partition by date_format(order_date, "%Y%m") order by round(sum(sale_price * quantity), 2) desc) as rn
from retail
group by category,months order by months) as a
where rn = 1),

c as(select * , row_number() over(partition by category order by sales desc) as r from b)

select category, months, sales from c where r = 1;"""

cursor.execute(query)
data = cursor.fetchall()

df = pd.DataFrame(data, columns = ["Category", "Month", "Sales"])
df

,Category,Month,Sales
0,Furniture,202308,230523.5
1,Office Supplies,202302,287244.6
2,Technology,202310,295586.5


In [9]:
# which sub category had highest growth by profit in 2023 compare to 2022

query = """with a as(select sub_category, round(sum(quantity * profit), 2) as 2022_profit
from retail
where year(order_date) = 2022
group by sub_category),

b as (select sub_category, round(sum(quantity * profit), 2) as 2023_profit
from retail
where year(order_date) = 2023
group by sub_category)

select sub_category,growth from
(select a.sub_category, round((b.2023_profit - a.2022_profit),2) as growth, 
dense_rank() over(order by round((b.2023_profit - a.2022_profit),2) desc) as rn
from a join b on a.sub_category = b.sub_category) as c
where rn = 1;"""

cursor.execute(query)
data = cursor.fetchall()

df = pd.DataFrame(data, columns = ["Sub_Category", "Growth"])
df

,Sub_Category,Growth
0,Machines,22334.3
